In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("data/unimelb_training.csv", header=0, dtype="str")
data = data.drop("Unnamed: 251", axis=1)
data["Year"] = data["Start.date"].map(lambda x: float(x.split("/")[2]))
data["Year"].dtype


numerical_data_feature_category = [
    ("RFCD.Percentage.", 5), 
    ("SEO.Percentage.", 5), 
    ("Year.of.Birth.", 15),
    ("Number.of.Successful.Grant.", 15),
    ("Number.of.Unsuccessful.Grant.", 15),
    ("A..", 15),
    ("A.", 15),
    ("B.", 15),
    ("C.", 15)
]

for feature_name, feature_range in numerical_data_feature_category:
    numerical_data_features = [feature_name+str(i) for i in range(1, feature_range+1)]
    for numerical_data_feature in numerical_data_features:
        data[numerical_data_feature] = data[numerical_data_feature].astype(float)

data.dtypes

Grant.Application.ID                        object
Grant.Status                                object
Sponsor.Code                                object
Grant.Category.Code                         object
Contract.Value.Band...see.note.A            object
Start.date                                  object
RFCD.Code.1                                 object
RFCD.Percentage.1                          float64
RFCD.Code.2                                 object
RFCD.Percentage.2                          float64
RFCD.Code.3                                 object
RFCD.Percentage.3                          float64
RFCD.Code.4                                 object
RFCD.Percentage.4                          float64
RFCD.Code.5                                 object
RFCD.Percentage.5                          float64
SEO.Code.1                                  object
SEO.Percentage.1                           float64
SEO.Code.2                                  object
SEO.Percentage.2               

In [2]:
# Contigency for persons

In [3]:
def process_column(data, prefix, nb_max=15):
    unique_values = set([])
    all_values = []
    for i in range(1, nb_max+1):
        unique_values.update(data[prefix+str(i)].unique())
        all_values.extend(list(data[prefix+str(i)]))
    return unique_values, all_values

In [21]:
unique_roles, all_roles = process_column(data, "Role.")

all_roles_tuple = tuple(unique_roles)
print(all_roles_tuple)
# print(len(all_roles)) # 8708*15
# print(sum([True if np.isreal(x) and np.isnan(x) else False for x in all_roles])/len(all_roles)) # this value is high because we concatenate all the 15 persons

# pd.Series(all_roles).value_counts()

(nan, 'CHIEF_INVESTIGATOR', 'STUD_CHIEF_INVESTIGATOR', 'EXT_CHIEF_INVESTIGATOR', 'PRINCIPAL_SUPERVISOR', 'HONVISIT', 'EXTERNAL_ADVISOR', 'DELEGATED_RESEARCHER', 'STUDRES')


In [45]:
def add_contigency(prefix, nb_max, unique_values):
    nb_unique_values = len(unique_values)
    has_nans = np.nan in unique_values
    nan_index = unique_values.index(np.nan)
    new_feature_indices = [prefix+str(value) for value in unique_values]
    def nested_func(x):
        values_frequency = [0]*nb_unique_values
        for i in range(1, nb_max+1):
            values_frequency[unique_values.index(x[prefix+str(i)])] += 1
        if has_nans:
            values_frequency[nan_index] = 0
        summe = sum(values_frequency)
        if summe != 0:
            return pd.Series([value/summe for value in values_frequency], index=new_feature_indices)
        else:
            return values_frequency
    return nested_func

aaa = data.apply(add_contigency("Role.", 15, all_roles_tuple), axis=1, raw=True)

In [31]:
np.nan in (1,2,3,nan)

NameError: name 'nan' is not defined

In [34]:
sum([1,2,3])

6